In [1]:
import os #paths to file
import numpy as np # linear algebra
import pandas as pd # data processing
import warnings# warning filter
import scipy as sp #pivot egineering

from sklearn.metrics.pairwise import cosine_similarity

pd.set_option("display.max_rows", None, "display.max_columns", None)

warnings.filterwarnings("always")
warnings.filterwarnings("ignore")

In [2]:
os.getcwd()

'/Users/michealalexander/Desktop/anime_files'

In [3]:
anime_df = pd.read_csv('/Users/michealalexander/Desktop/anime_files/anime.csv')
rating_df = pd.read_csv('/Users/michealalexander/Desktop/anime_files/rating.csv')

In [4]:
print(anime_df.info())
print(rating_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB
None


anime.csv:
- anime_id: unique identifier for the anime
- name: name of the anime
- genre: list of genres of the anime 
- type: denotes whether the anime is a TV show or movie
- episodes: number of episodes in the anime series
- rating: average rating for the anime 
- members: number of community members belonging to the anime group

ratings.csv:
- user id: unique identifier for each user
- anime id: unique identifier for the anime
- rating: user's rating of the anime 

In [5]:
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [6]:
anime_df.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

I'll just remove entries that don't have a specified type since genre and the rating column from this dataframe are not necessary for the current analysis.

In [7]:
anime_df.dropna(subset=['type'],inplace=True)

In [10]:
anime_df.isnull().sum()

anime_id      0
name          0
genre        59
type          0
episodes      0
rating      205
members       0
dtype: int64

In [8]:
rating_df.head(10)

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
5,1,355,-1
6,1,356,-1
7,1,442,-1
8,1,487,-1
9,1,846,-1


I'm assuming that a rating of -1 means the user did not enter a rating for the anime, and will remove these instances.

In [11]:
rating_df.rating.unique()

array([-1, 10,  8,  6,  9,  7,  3,  5,  4,  1,  2])

In [12]:
rating_df.replace(-1, np.nan, inplace=True)
rating_df.isnull().sum()

user_id           0
anime_id          0
rating      1476496
dtype: int64

In [13]:
rating_df.dropna(subset=['rating'], inplace=True)

In [14]:
rating_df.isnull().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

In [15]:
anime_df = anime_df[anime_df['type'] == 'TV'] # only looking for anime series recommendations

In [16]:
anime_df.head(10)

,anime_id,name,genre,type,episodes,rating,members
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855
9,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13,9.11,81109
10,4181,Clannad: After Story,"Drama, Fantasy, Romance, Slice of Life, Supern...",TV,24,9.06,456749
12,918,Gintama,"Action, Comedy, Historical, Parody, Samurai, S...",TV,201,9.04,336376
13,2904,Code Geass: Hangyaku no Lelouch R2,"Action, Drama, Mecha, Military, Sci-Fi, Super ...",TV,25,8.98,572888


In [17]:
combined_df = rating_df.merge(right=anime_df, right_on='anime_id', left_on='anime_id')
# merge the dataframes on anime_id

In [18]:
combined_df.head(30)

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
0,1,8074,10.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
1,3,8074,6.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
2,5,8074,2.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
3,12,8074,6.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
4,14,8074,6.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
5,17,8074,7.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
6,24,8074,7.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
7,27,8074,9.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
8,29,8074,2.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
9,30,8074,8.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892


In [19]:
combined_df = combined_df[['user_id', 'rating_x', 'name']]
# remove irrelevant columns

In [20]:
combined_df.head(10)

,user_id,rating_x,name
0,1,10.0,Highschool of the Dead
1,3,6.0,Highschool of the Dead
2,5,2.0,Highschool of the Dead
3,12,6.0,Highschool of the Dead
4,14,6.0,Highschool of the Dead
5,17,7.0,Highschool of the Dead
6,24,7.0,Highschool of the Dead
7,27,9.0,Highschool of the Dead
8,29,2.0,Highschool of the Dead
9,30,8.0,Highschool of the Dead


In [21]:
combined_df.shape

(4364294, 3)

In [22]:
final_10000_users = combined_df[combined_df['user_id'] < 10000]

In [23]:
final_10000_users.shape

(582364, 3)

In [24]:
final_10000_users.head(10)

,user_id,rating_x,name
0,1,10.0,Highschool of the Dead
1,3,6.0,Highschool of the Dead
2,5,2.0,Highschool of the Dead
3,12,6.0,Highschool of the Dead
4,14,6.0,Highschool of the Dead
5,17,7.0,Highschool of the Dead
6,24,7.0,Highschool of the Dead
7,27,9.0,Highschool of the Dead
8,29,2.0,Highschool of the Dead
9,30,8.0,Highschool of the Dead


In [25]:
piv_table = final_10000_users.pivot_table(index='name', values='rating_x', columns='user_id')

In [27]:
pivot_sparse = sp.sparse.csr_matrix(piv_table.fillna(0))

In [29]:
similarities_df = cosine_similarity(pivot_sparse)

In [30]:
recommender_df = pd.DataFrame(similarities_df, columns = piv_table.index, index = piv_table.index)

In [65]:
recommender_df.head(10)

name                                          .hack//Roots  .hack//Sign  \
name                                                                      
.hack//Roots                                      1.000000     0.548626   
.hack//Sign                                       0.548626     1.000000   
.hack//Tasogare no Udewa Densetsu                 0.512911     0.574565   
009-1                                             0.063583     0.052140   
07-Ghost                                          0.146033     0.135908   
11eyes                                            0.173811     0.148782   
12-sai.: Chicchana Mune no Tokimeki               0.037166     0.044753   
3 Choume no Tama: Uchi no Tama Shirimasenka?      0.000000     0.000000   
30-sai no Hoken Taiiku                            0.075010     0.084560   
91 Days                                           0.042923     0.047144   

name                                          .hack//Tasogare no Udewa Densetsu  \
name                                                                              
.hack//Roots                                                           0.512911   
.hack//Sign                                                            0.574565   
.hack//Tasogare no Udewa Densetsu                                      1.000000   
009-1                                                                  0.079526   
07-Ghost                                                               0.123300   
11eyes                                                                 0.141542   
12-sai.: Chicchana Mune no Tokimeki                                    0.041844   
3 Choume no Tama: Uchi no Tama Shirimasenka?                           0.000000   
30-sai no Hoken Taiiku                                                 0.056021   
91 Days                                                                0.036726   

name                                             009-1  07-Ghost    11eyes  \
name                                                                         
.hack//Roots                                  0.063583  0.146033  0.173811   
.hack//Sign                                   0.052140  0.135908  0.148782   
.hack//Tasogare no Udewa Densetsu             0.079526  0.123300  0.141542   
009-1                                         1.000000  0.051764  0.064639   
07-Ghost                                      0.051764  1.000000  0.291585   
11eyes                                        0.064639  0.291585  1.000000   
12-sai.: Chicchana Mune no Tokimeki           0.055766  0.025846  0.026559   
3 Choume no Tama: Uchi no Tama Shirimasenka?  0.000000  0.023791  0.018709   
30-sai no Hoken Taiiku                        0.014178  0.076301  0.151211   
91 Days                                       0.026562  0.095925  0.101275   

name                                          12-sai.: Chicchana Mune no Tokimeki  \
name                                                                                
.hack//Roots                                                             0.037166   
.hack//Sign                                                              0.044753   
.hack//Tasogare no Udewa Densetsu                                        0.041844   
009-1                                                                    0.055766   
07-Ghost                                                                 0.025846   
11eyes                                                                   0.026559   
12-sai.: Chicchana Mune no Tokimeki                                      1.000000   
3 Choume no Tama: Uchi no Tama Shirimasenka?                             0.000000   
30-sai no Hoken Taiiku                                                   0.085000   
91 Days                                                                  0.160117   

name                                          3 Choume no Tama: Uchi no Tama Shirimasenka?  \
name                                              

In [94]:
anime = pd.DataFrame(recommender_df.index) # create a dataframe that has all names of the anime that recommendations can be based on

In [98]:
anime

,name
0,.hack//Roots
1,.hack//Sign
2,.hack//Tasogare no Udewa Densetsu
3,009-1
4,07-Ghost
5,11eyes
6,12-sai.: Chicchana Mune no Tokimeki
7,3 Choume no Tama: Uchi no Tama Shirimasenka?
8,30-sai no Hoken Taiiku
9,91 Days


In [97]:
def recommend_anime(anime_title):
    print('Based on {}, you might enjoy these suggestions:\n'.format(anime_title))
    for anime in recommender_df.sort_values(by = anime_title, ascending=False).index[1:6]:
        print('{}, {}% match'.format(anime, round(recommender_df[anime][anime_title]*100, 2)))

Anime Recommendations for series that I enjoy:

In [87]:
recommend_anime('Shingeki no Kyojin')

Based on Shingeki no Kyojin, you might enjoy these suggestions:

Sword Art Online, 67.37% match
Tokyo Ghoul, 59.66% match
Mirai Nikki (TV), 59.05% match
Death Note, 57.64% match
No Game No Life, 57.1% match


In [88]:
recommend_anime('Haikyuu!!')

Based on Haikyuu!!, you might enjoy these suggestions:

Haikyuu!! Second Season, 70.46% match
Kuroko no Basket 2nd Season, 51.29% match
Kuroko no Basket, 49.78% match
Kuroko no Basket 3rd Season, 48.55% match
Gekkan Shoujo Nozaki-kun, 43.83% match


In [99]:
recommend_anime('Boku no Hero Academia')

Based on Boku no Hero Academia, you might enjoy these suggestions:

One Punch Man, 53.7% match
Koutetsujou no Kabaneri, 51.41% match
Re:Zero kara Hajimeru Isekai Seikatsu, 50.72% match
Boku dake ga Inai Machi, 50.2% match
Kiznaiver, 48.78% match


In [100]:
recommend_anime('Re:Zero kara Hajimeru Isekai Seikatsu')

Based on Re:Zero kara Hajimeru Isekai Seikatsu, you might enjoy these suggestions:

Boku dake ga Inai Machi, 55.02% match
Kono Subarashii Sekai ni Shukufuku wo!, 54.22% match
Koutetsujou no Kabaneri, 51.51% match
Boku no Hero Academia, 50.72% match
Hai to Gensou no Grimgar, 50.58% match
